In [1]:
# Instalation des libs
# !pip install pandas sqlalchemy binance-python nest_asyncio TA-Lib

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from binance import ThreadedWebsocketManager
from binance.client import Client
import json
from functools import partial
import nest_asyncio
nest_asyncio.apply()
from func import *

---

In [3]:
# Récupération des Cle API dans le fichier config json
with open("config.json") as config_file:
    config = json.load(config_file)
    api_key = config["API_KEY"]
    api_secret = config["API_SECRET"]
    
# Init du client python-binance
client = Client(api_key, api_secret)

In [4]:
# Symbol & interval
SYMBOL = 'ETHUSDT'
INTERVAL = '1m'

# Connection à la bdd
engine = create_engine(f'sqlite:///{SYMBOL}.db')

In [6]:
def format_message_to_dataframe(msg):
    # Créer un dictionnaire avec les données extraites
    data = {'open': [msg['k']['o']],
            'high': [msg['k']['h']],
            'low': [msg['k']['l']],
            'close': [msg['k']['c']],
            'volume': [msg['k']['v']],
            'close_time': [msg['k']['T']]}

    # Créer un dataframe à partir du dictionnaire
    df = pd.DataFrame(data)
    
    # Convertir en datetime et définir close_time comme index
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    df.set_index('close_time', inplace=True)
    
    # Convertir les données en float
    df = df.astype(float)
    
    return df

def process_candle_data(msg, symbol):
    kline = msg['k']  # Extraire les données du kline
    is_closed = kline['x']  # Vérifier si la bougie est fermée
    
    if is_closed:
        print('# NEW CANDLE')
        
        # Formater les données
        df_ohlcv = format_message_to_dataframe(msg)
        print(df_ohlcv.head(1))
        
        # Enregistrer les données dans la base #DataWareHouse
        df_ohlcv.to_sql(symbol, con=engine, if_exists='append', index=True)
        
        # Traiter les données
        process_data()

def start_candlestick_socket(symbol, interval):
    print('### DÉBUT ÉCOUTE WEBSOCKET')
    twm = ThreadedWebsocketManager()
    twm.start()

    # Utiliser functools.partial pour passer le symbole en argument
    twm.start_kline_socket(callback=partial(process_candle_data, symbol=symbol), symbol=symbol, interval=interval)

    # Garder le WebSocket actif
    twm.join()

In [7]:
def process_data():
    ### Récupérer les données depuis la base
    df = pd.read_sql(SYMBOL, con=engine)
    
    # Vérifier qu'il y a assez de données pour traiter
    rolling_window = 20
    if len(df) < rolling_window:
        print('> WARMUP')
        return
    
    print('# TRAITEMENT DES DATA')
    
    ### Strategie Processing ###
    df = add_technical_indicator(df)
    df = add_signals(df)
    
    ### Risk Management ###
    if (df['signal'].iloc[-1] == 1) and (has_open_orders(client, SYMBOL) == False):  # Si signal et aucune position en cours
        # Générer les paramètres du trade
        entry_price = df['close'].iloc[-1]
        SL, TP = generate_TP_SL(entry_price, SL_pct=0.01, risk_reward_ratio=2)
        
    ### Order Management ###
        # Passer un ordre au marché
        qty_bought = place_market_order(client, SYMBOL, side='BUY', quantity=1)
        # Placer un ordre OCO (TP+SL)
        place_oco_order(client, SYMBOL, side='SELL', quantity=qty_bought, TP=TP, SL=SL, limit_pct=0.005)

---

In [ ]:
# Start the WebSocket
start_candlestick_socket(SYMBOL, INTERVAL)